In [ ]:
import requests
import json
import pandas as pd
import csv
import time

#APIKEY = 

In [ ]:
def request_places(lat, lon, radius, api_key,
                 place_type=None, page_token=None, verbose=True):
    prefix = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    options = [
        'location={lat},{lon}'.format(lat=lat, lon=lon),
        'radius={r}'.format(r=radius),
        'key={k}'.format(k=api_key),
    ]
    
    if place_type is not None and not isinstance(place_type, str):
        raise ValueError('place_type should be string')
    
    if place_type is not None:
        options.append(
            'type={t}'.format(t=place_type)
        )
    
    if page_token is not None:
        options.append(
            'pagetoken={t}'.format(t=page_token)
        )
    
    url = prefix + '&'.join(options)
    if verbose:
        print(url)
        
    response = requests.get(url)
    res = json.loads(response.text)
    return res

In [ ]:
def parse_places(lat, lon, radius, api_key,
                 place_type=None, page_token=None, verbose=True):
    places = []
    r = request_places(lat, lon, radius, api_key,
                       place_type=place_type, page_token=page_token, verbose=verbose)
    
    if r['status'] != 'OK':
        return places
    
    places.extend(r['results'])
    while True:
        page_token = r.get("next_page_token", None)
        if r['status'] != 'OK' or page_token is None:
            break
            
        time.sleep(5)
        r = request_places(lat, lon, radius, api_key, 
                           place_type=place_type, page_token=page_token, verbose=verbose)
        places.extend(r['results'])
    
    return places

In [ ]:
filename = "臺北市立美術館"

places = parse_places(
    25.0724118,121.5226215,
    1000,
    APIKEY,
    place_type='restaurant'
)

In [ ]:
lats = [i['geometry']['location']['lat'] for i in places]
lons = [i['geometry']['location']['lng'] for i in places]
names = [i['name'] for i in places]
place_ids = [i['place_id'] for i in places]
ratings = [i.get('rating', 0) for i in places]

In [ ]:
#Place Types : https://developers.google.com/maps/documentation/places/web-service/supported_types

In [ ]:
for i, j, k, l, m in zip(lats, lons, names, place_ids, ratings):
    print (i, j, k, l, m)

In [ ]:
url = []
for i in range(len(lats)):
    wc_url = "https://www.google.com/maps/search/?api=1&query={lats},{lons}&query_place_id={place_ids}".format(lats=lats[i], lons=lons[i], place_ids=place_ids[i])
    url.append(wc_url)
        

In [ ]:
df = pd.DataFrame ({"names": names,
                    "lats": lats, 
                    "lons": lons, 
                    "place_ids": place_ids, 
                    "ratings": ratings})

df.to_csv(filename+".csv", index= None, encoding = "utf8")

df = pd.DataFrame ({"names": names,
                    "lats": lats, 
                    "lons": lons, 
                    "place_ids": place_ids, 
                    "ratings": ratings,
                    "url": url})
df.to_excel(filename+".xlsx", index= None)